# PyCity Schools Analysis

- Type your analysis


In [594]:
# Import appropriate modules

import pandas as pd
from pathlib import Path

# Read the csv files and merge them in to a DataFrame

school_data_path = Path("Resources/schools_complete.csv")
student_data_path = Path("Resources/students_complete.csv")

# Setup separate DataFrames for school and student data
school_data = pd.read_csv(school_data_path)
student_data = pd.read_csv(student_data_path)

# Merge the DataFrames by school name
school_student_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_student_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


# District Summary


#### Total unique school count
---

In [595]:
# Find the total number of unique schools

school_count = len(school_student_df["school_name"].unique())
school_count

15

#### Total Student Count
---

In [596]:
# Find the total number of students

student_count = len(school_student_df["student_name"])
student_count

39170

#### Total Budget
---

In [597]:
# Find the total budget

total_budget = school_data["budget"].sum()
total_budget

24649428

#### Total Average Math Score
----

In [598]:
# Find the average math score

average_math_score = round(school_student_df["math_score"].mean(), 2)
average_math_score

78.99

#### Total Average Reading Score
---

In [599]:
# Find the average reading score

average_reading_score = round(school_student_df["reading_score"].mean(), 2)
average_reading_score

81.88

#### Total Percentage of Students Passing Math
----

In [600]:
# Find percentage of students passing math. Passing grade is 70 or higher

math_pass = school_student_df.loc[(school_student_df["math_score"]) >= 70, :]["math_score"].count()
per_math_pass = round(math_pass / student_count * 100, 2)
per_math_pass

74.98

#### Total Percentage of Students Passing Reading
----

In [601]:
# Find percentage of students passing reading. Passing grade is 70 or higher

read_pass = school_student_df.loc[(school_student_df["reading_score"]) >= 70, :]["reading_score"].count()
per_read_pass = round(read_pass / student_count * 100, 2)
per_read_pass

85.81

#### Total Percentage of Students Passing Math and Reading
----

In [602]:
# Find the percentage of students who passed math and reading

overall_pass = school_student_df.loc[((school_student_df["reading_score"]) >= 70) & 
                                     ((school_student_df["math_score"]) >= 70), :]["reading_score"].count()

per_overall_pass = round(overall_pass / student_count * 100, 2)

per_overall_pass

65.17

### District key metrics
---

In [603]:
# Create DataFrame for district's key metrics

district_summary = pd.DataFrame([{"Total Schools": school_count, "Total Students": student_count, "Total Budget": total_budget,
                                  "Average Math Score": average_math_score, "Average Reading Score": average_reading_score,
                                  "% Passing Math": per_math_pass, "%Passing Reading": per_read_pass, "% Overall Passing": per_overall_pass}])

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,%Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,65.17


# School Summary
---

In [604]:
# We gather these as they wil be useful later when we do loops
# Create a list of schools

school_list = school_data["school_name"].unique()

# Create a new school_data DataFrame with index as school_name

school_data_new = school_data.set_index("school_name")

#### School type
----

In [613]:
# Create dictionaries to store data for the school type per school. Keys are the school names, values are school type
school_type = {"school_name": [], "type": []}

# Loop through school_data store in the school_type dictionary

for school in school_list:
    school_type["school_name"].append(school)
    school_type["type"].append(school_data_new.at[school, "type"])

# Store it in a DataFrame with school names as index, sorted in alphabetical order
school_type = pd.DataFrame(school_type).sort_values("school_name")
school_type = school_type.reset_index(drop = True)

school_type.head()


,school_name,type
0,Bailey High School,District
1,Cabrera High School,Charter
2,Figueroa High School,District
3,Ford High School,District
4,Griffin High School,Charter


#### Total Students
---

In [614]:
# Lists the number of students in each school using value_counts

per_school_stu = school_student_df["school_name"].value_counts()
per_school_stu = per_school_stu.to_frame()
per_school_stu = per_school_stu.reset_index()
per_school_stu = per_school_stu.rename(columns={"school_name": "total_students",
                                                 "index": "school_name"})

per_school_stu.head()


,school_name,total_students
0,Bailey High School,4976
1,Johnson High School,4761
2,Hernandez High School,4635
3,Rodriguez High School,3999
4,Figueroa High School,2949


#### Budget per School
---

In [615]:
# Create dictionaries to store data for the budget per school.
# and budget per capita respectively
per_school_budget = {"school_name": [], "budget": []}              # Dictionary for budget
per_school_capita = {"school_name": [], "budget_per_capita": []}   # Dictionary for budget per capita


# Loop through school_data and add to the school budget and school budget per capita 
for school in school_list:
    per_school_budget["school_name"].append(school)
    per_school_budget["budget"].append(school_data_new.at[school, "budget"])
    per_school_capita["school_name"].append(school)
    per_school_capita["budget_per_capita"].append(int(school_data_new.at[school, "budget"]) / \
                                       int(school_data_new.at[school, "size"]))

# Create a DataFrame for each of budget per schoool and budget per capita per school
per_school_budget = pd.DataFrame(per_school_budget)

per_school_capita = pd.DataFrame(per_school_capita)

per_school_budget.head()
per_school_capita.head()

,school_name,budget_per_capita
0,Huang High School,655.0
1,Figueroa High School,639.0
2,Shelton High School,600.0
3,Hernandez High School,652.0
4,Griffin High School,625.0


#### Average math score
---

In [616]:
# Group the data by the school name
school_summary_df = school_student_df.groupby("school_name")

In [617]:
# Find average math score for each school

per_school_av_math = school_summary_df[["math_score"]].mean()
per_school_av_math = per_school_av_math.reset_index()
per_school_av_math.head()

,school_name,math_score
0,Bailey High School,77.048432
1,Cabrera High School,83.061895
2,Figueroa High School,76.711767
3,Ford High School,77.102592
4,Griffin High School,83.351499


#### Average reading score
---

In [618]:
# Find average reading score for each school
per_school_av_read = school_summary_df[["reading_score"]].mean()
per_school_av_read = per_school_av_read.reset_index()
per_school_av_read.head()

,school_name,reading_score
0,Bailey High School,81.033963
1,Cabrera High School,83.975780
2,Figueroa High School,81.158020
3,Ford High School,80.746258
4,Griffin High School,83.816757


#### % Passing Math
---

In [622]:
# We compute the number of students passing math per school
per_school_math_pass = {"school_name": [], "%_pass_math": []}

for school in school_list:
    per_school_math_pass["school_name"].append(school)
    per_school_math_pass["%_pass_math"].append(int(school_student_df.loc[(school_student_df["school_name"] == school) &
                                                                         (school_student_df["math_score"] >= 70), :].count()[1]) / \
                                                                            school_data_new.at[school, "size"] * 100)

per_school_math_pass = pd.DataFrame(per_school_math_pass)

per_school_math_pass


,school_name,%_pass_math
0,Huang High School,65.683922
1,Figueroa High School,65.988471
2,Shelton High School,93.867121
3,Hernandez High School,66.752967
4,Griffin High School,93.392371
5,Wilson High School,93.867718
6,Cabrera High School,94.133477
7,Bailey High School,66.680064
8,Holden High School,92.505855
9,Pena High School,94.594595


#### % Passing Reading
---

In [623]:
# We compute the number of students passing reading per school
per_school_read_pass = {"school_name": [], "%_pass_read": []}

for school in school_list:
    per_school_read_pass["school_name"].append(school)
    per_school_read_pass["%_pass_read"].append(int(school_student_df.loc[(school_student_df["school_name"] == school) &
                                                                         (school_student_df["reading_score"] >= 70), :].count()[1]) / \
                                                                            school_data_new.at[school, "size"] * 100)

per_school_read_pass = pd.DataFrame(per_school_read_pass)

per_school_read_pass

,school_name,%_pass_read
0,Huang High School,81.316421
1,Figueroa High School,80.739234
2,Shelton High School,95.854628
3,Hernandez High School,80.862999
4,Griffin High School,97.138965
5,Wilson High School,96.539641
6,Cabrera High School,97.039828
7,Bailey High School,81.933280
8,Holden High School,96.252927
9,Pena High School,95.945946


#### % Overall Pass
---

In [628]:
per_school_overall_pass = {"school_name": [], "%_pass_overall": []}

for school in school_list:
    per_school_overall_pass["school_name"].append(school)
    per_school_overall_pass["%_pass_overall"].append(int(school_student_df.loc[((school_student_df["school_name"] == school) &
                                                                         (school_student_df["reading_score"] >= 70)) & 
                                                                         (school_student_df["math_score"] >= 70), :].count()[1]) / \
                                                                            school_data_new.at[school, "size"] * 100)

per_school_overall_pass = pd.DataFrame(per_school_overall_pass)

per_school_overall_pass

,school_name,%_pass_overall
0,Huang High School,53.513884
1,Figueroa High School,53.204476
2,Shelton High School,89.892107
3,Hernandez High School,53.527508
4,Griffin High School,90.599455
5,Wilson High School,90.582567
6,Cabrera High School,91.334769
7,Bailey High School,54.642283
8,Holden High School,89.227166
9,Pena High School,90.540541


#### Merge the per school data into a Dataframe
----

In [634]:
# Merging all the DataFrames into one that summarizes the key metrics per school

per_school_summary = pd.DataFrame.merge(school_type, per_school_stu, on="school_name", how="left")
per_school_summary = pd.DataFrame.merge(per_school_summary, per_school_budget, on="school_name", how="left")
per_school_summary = pd.DataFrame.merge(per_school_summary, per_school_capita, on="school_name", how="left")
per_school_summary = pd.DataFrame.merge(per_school_summary, per_school_av_math, on="school_name", how="left")
per_school_summary = pd.DataFrame.merge(per_school_summary, per_school_av_read, on="school_name", how="left")
per_school_summary = pd.DataFrame.merge(per_school_summary, per_school_math_pass, on="school_name", how="left")
per_school_summary = pd.DataFrame.merge(per_school_summary, per_school_read_pass, on="school_name", how="left")
per_school_summary = pd.DataFrame.merge(per_school_summary, per_school_overall_pass, on="school_name", how="left")

# Formatting

# Rename the column names
per_school_summary = per_school_summary.rename(columns={"school_name": "School", "type": "School Type", "total_students": "Total Students",
                                                "budget": "Total School Budget", "budget_per_capita": "Per Student Budget",
                                                "math_score": "Average Math Score", "reading_score": "Average Reading Score",
                                                "%_pass_math": "% Passing Math", "%_pass_read": "% Passing Reading", 
                                                "%_pass_overall": "% Overall Passing"})

# Make the index as the school_name
per_school_summary = per_school_summary.set_index("School")

per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)
per_school_summary




,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
